In [1]:
import pandas as pd
import numpy as np 
import keras 
from keras.layers import Dense, Embedding, LSTM, Input
import matplotlib.pyplot as plt 
from keras.models import Model
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
ls

drive/  sample_data/


In [0]:
import os
os.chdir('drive/My Drive/DataDays/second_round')

# Preprocess


In [0]:
data = pd.read_csv('./divar_posts_dataset.csv')

In [6]:
print(len(data.cat1.unique()))
data.cat1.unique()


6


array(['for-the-home', 'vehicles', 'personal', 'electronic-devices',
       'businesses', 'leisure-hobbies'], dtype=object)

In [0]:
print(len(data.cat2.unique()))
data.cat2.unique()


28


array(['furniture-and-home-decore', 'cars', 'baby-and-toys',
       'parts-accessories', 'utensils-and-appliances',
       'clothing-and-shoes', 'mobile-tablet',
       'childrens-clothing-and-shoe', 'game-consoles-and-video-games',
       'audio-video', 'building-and-garden', 'jewelry-and-watches',
       'equipments-and-machinery', 'bicycle', 'animals', nan, 'batch',
       'musical-instruments', 'health-beauty', 'motorcycles', 'computers',
       'sport-leisure', 'book-student-literature', 'utility',
       'travel-packages', 'hobby-collectibles', 'leisure-hobbies-toys',
       'phone'], dtype=object)

In [0]:
print(len(data.cat3.unique()))
data.cat3.unique()


67


array(['sofa-armchair', 'antiques-and-art', 'heavy', 'personal-toys', nan,
       'cookware-tableware', 'light', 'clothing', 'mobile-phones',
       'tv-projector', 'garden-and-patio', 'watches', 'offices',
       'kitchen', 'farm-animals', 'cat', 'video-dvdplayer',
       'shoes-belt-bag', 'industrial', 'tv-and-stereo-furniture', 'birds',
       'guitar-bass-amplifier', 'beds-bedroom', 'carpets',
       'mobile-tablet-accessories', 'fridge-and-freezer', 'lighting',
       'tables-and-chairs', 'strollers-and-accessories',
       'modem-and-network-equipment', 'jewelry', 'stereo-surround',
       'camera-camcoders', 'training', 'storage', 'stove-and-heating',
       'barbershop-and-beautysalon', 'dishwasher',
       'parts-and-accessories', 'cafe-and-restaurant', 'microwave-stove',
       'washer-dryer', 'educational', 'childrens-furniture',
       'piano-keyboard', 'desktops', 'shop-and-cash', 'laptops',
       'rhinestones', 'bathroom-wc-sauna', 'mp3-player',
       'textile-ornaments

In [0]:
data['cat1_2'] = data.cat1.astype(str).str.cat(data.cat2.astype(str), sep='|')
data['cat1_2_3'] = data.cat1_2.astype(str).str.cat(data.cat3.astype(str), sep='|')


In [0]:
train,test = train_test_split(data, random_state = 100)

In [0]:
labels = {j:i for i,j in enumerate(train.cat1_2_3.unique())}

In [0]:
X_train = train.apply(lambda i : i.title + ' ' + i.desc, axis=1)
Y_train = train.apply(lambda i : labels[i.cat1_2_3], axis=1)

In [0]:
X_test = test.apply(lambda i : i.title + ' ' + i.desc, axis=1)
Y_test = test.apply(lambda i : labels[i.cat1_2_3], axis=1)

In [12]:
MAX_NB_WORDS = 200000

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, char_level=False)
tokenizer.fit_on_texts(X_test)
sequences = tokenizer.texts_to_sequences(X_train)
sequences_test = tokenizer.texts_to_sequences(X_test)
word_index = tokenizer.word_index

print('Found %s unique tokens.' % len(word_index))

Found 200326 unique tokens.


In [0]:
# index_to_word = dict((i, w) for w, i in tokenizer.word_index.items())

In [0]:
MAX_SEQUENCE_LENGTH = 200 
x_train = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
x_test = pad_sequences(sequences_test, maxlen=MAX_SEQUENCE_LENGTH)


Y_train = to_categorical(np.asarray(Y_train))
Y_test = to_categorical(np.asarray(Y_test))


In [17]:
from keras.layers import Dense, Input, Flatten,Bidirectional
from keras.layers import GlobalAveragePooling1D, Embedding
from keras.models import Model
from keras.layers.embeddings import Embedding

EMBEDDING_DIM = 100
N_CLASSES = 87

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedding_layer = Embedding(MAX_NB_WORDS, EMBEDDING_DIM,
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)
embedded_sequences = embedding_layer(sequence_input)

x = Bidirectional(LSTM(256))(embedded_sequences)

predictions = Dense(N_CLASSES, activation='softmax')(x)


model = Model(sequence_input, predictions)


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 200)               0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 200, 100)          20000000  
_________________________________________________________________
bidirectional_5 (Bidirection (None, 512)               731136    
_________________________________________________________________
dense_3 (Dense)              (None, 87)                44631     
Total params: 20,775,767
Trainable params: 20,775,767
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(x_train, Y_train, validation_data=[x_test,Y_test],
          nb_epoch=10, batch_size=256)

Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 710726 samples, validate on 236909 samples
Epoch 1/10
  1280/710726 [..............................] - ETA: 1:19:58 - loss: 4.4228 - acc: 0.0992

In [0]:
model.fit(x_train, Y_train, validation_data=[x_test,Y_test],
          nb_epoch=10, batch_size=256)

In [0]:
model.save('./2dconvmodel4')

In [0]:
model.save('./2dconvmodel5')

In [0]:
from sklearn.metrics import roc_auc_score
output_test = model.predict(x_test)


In [0]:
from sklearn.metrics import classification_report
Y_test = np.argmax(Y_test,axis=1)
output_test = np.argmax(output_test,axis=1)
print(classification_report(Y_test, output_test))

In [0]:
tst = pd.read_csv('dataset.csv')

In [0]:
X_tst = tst.apply(lambda i : str(i.title) + ' ' + str(i.desc), axis=1)


In [0]:
texts_tst = X_tst.astype(str)


In [0]:
sequences_tst = tokenizer.texts_to_sequences(texts_tst)


In [0]:
MAX_SEQUENCE_LENGTH = 200

x_tst = pad_sequences(sequences_tst, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data test tensor:', x_tst.shape)

In [0]:
output_test = model.predict(x_tst)


In [0]:
pred=np.argmax(output_test,axis=1)

In [0]:
tst['cat1_2_3']=pred

In [0]:
merged = {}
for key, value in labels.items():
    merged[value] = key


In [0]:
tst['cat1_2_3'] = tst['cat1_2_3'].map(merged)

In [0]:
cat_tmp = tst['cat1_2_3'].str.split("|",n=1,expand=True)
tst['cat1'] = cat_tmp[0]
tst['cat2_3']=cat_tmp[1] 
cat2_tmp = tst['cat2_3'].str.split("|",n=1,expand=True)
tst['cat2'] = cat2_tmp[0]
tst['cat3']=cat2_tmp[1] 

In [0]:
tst = tst.drop(columns=['Unnamed: 0', 'archive_by_user', 'city', 'created_at', 'desc', 'id',
       'image_count', 'platform', 'price', 'title', 'cat1_2_3',
       'cat2_3',],axis=1)


In [0]:
tst.replace(['nan'], '', inplace=True)

In [0]:
tst.to_csv("conv2d_deep_em.csv")

In [0]:
model.save('./con2davalshodim_em')